In [ ]:
!pip install pyhrv

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

In [12]:
#LOAD ECG DATA
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp

convert_sr = False
task = dr.Tasks.BASELINE
data_type = dr.DataTypes.ECG
phase = dr.Phases.BASE_REST
fs = 250.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]

# ha_ecg, la_ecg = preprocessing.load_data(task, data_type, phase, convert_sr)

data = la_ecg[0]
print(data.shape)
print(data.iloc[0, 0])
# preprocessing.plot_1d_data(data[500:1000])
# data = data.to_numpy().flatten()
# working_data, measures = hp.process_segmentwise(data, fs, segment_width=55, segment_overlap=5/55)
# print(measures.keys())

# ha_ecg = preprocessing.crop_list_of_ndarrays(ha_ecg)
# la_ecg = preprocessing.crop_list_of_ndarrays(la_ecg)
# freq, ha_ecg_fft = preprocessing.calculate_fft(ha_ecg, fs, f_dim)
# freq, la_ecg_fft = preprocessing.calculate_fft(la_ecg, fs, f_dim)

(39242, 2)
10.0


In [ ]:
# EXAMPLES ON HOW TO PLOT FFT USING DIFFERENT METHODS (calculate_fft, calculate_fft_1d, calculate_group_metric)

importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

convert_sr = False
is_clean_ecg = False
task = dr.Tasks.BASELINE
data_type = dr.DataTypes.ECG
phase = dr.Phases.BASE_REST
fs = 250.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]

ha_ecg, la_ecg = preprocessing.load_data(task, data_type, phase, convert_sr)
data = ha_ecg[0]
data = np.reshape(data, (data.size, 1))
freq, fft = preprocessing.calculate_fft(data, fs, feature_dim=f_dim)
dt.plot_fft_data(freq[1:], fft[0][1:], data_type="ECG BASELINE REST", labels="HA")



freq, fft = preprocessing.calculate_fft_1d(data, fs)
dt.plot_fft_data(freq[1:], fft[1:], data_type="ECG BASELINE REST", labels="HA")


print("FROM BATCH CALCULATED FFT")
freq, ha_ecg_fft = preprocessing.calculate_group_metric("HA", task, data_type, phase, "fft")
dt.plot_fft_data(freq[1:], ha_ecg_fft[1:, 0, 0], data_type="ECG BASELINE REST", labels="HA")



In [2]:
# FEATURE EXTRACTION - BPM
# WORKS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

from biosppy.signals import ecg
import heartpy as hp

convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
is_clean_ecg = False
data_type = dr.DataTypes.ECG
metrics = "bpm"
fs = 250.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK]
    for phase in phases:
        ha_ecg, la_ecg = preprocessing.load_data(task, data_type, phase, convert_sr, is_clean_ecg)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_ecg)):
            try:
                subject = la_ecg[i].iloc[0, 0]
                data = la_ecg[i].iloc[:, -1].to_numpy().flatten()
                if data.size == 0:
                    print(f"LA index {i} has value None")
                    bpm = [0.0]
                else:
                    bpm = preprocessing.get_bpm_biosppy(data)
            except Exception as e:
                print(f"LA index {i} has value None")
                bpm = [0.0]

            bpm.insert(0, int(subject))
            la.append(bpm)

        print(f"HA {phase}")
        for i in range(len(ha_ecg)):
            try:
                subject = ha_ecg[i].iloc[0, 0]
                data = ha_ecg[i].iloc[:, -1].to_numpy().flatten()
                if data.size == 0:
                    print(f"HA index {i} has value None")
                    bpm = [0.0]
                else:
                    bpm = preprocessing.get_bpm_biosppy(data)
            except Exception as e:
                print(f"HA index {i} has value None")
                bpm = [0.0]

            bpm.insert(0, int(subject))
            ha.append(bpm)

        # bpm_array = np.array([la, ha]).transpose()
        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{metrics}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{metrics}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


LA Baseline_Rest
HA Baseline_Rest
LA BugBox_Relax
HA BugBox_Relax
LA BugBox_Anticipate
HA BugBox_Anticipate
LA BugBox_Exposure
LA index 2 has value None
LA index 3 has value None
LA index 13 has value None
LA index 25 has value None
HA BugBox_Exposure
HA index 1 has value None
HA index 2 has value None
HA index 13 has value None
HA index 18 has value None
HA index 20 has value None
HA index 21 has value None
LA BugBox_Break
HA BugBox_Break
LA Speech_Relax
HA Speech_Relax
LA Speech_Anticipate
HA Speech_Anticipate
LA Speech_Exposure
HA Speech_Exposure
HA index 9 has value None
HA index 25 has value None
LA Speech_Break
HA Speech_Break


In [9]:
# FEATURE EXTRACTION WITH HEARTPY - RMSSD, SDNN
# WORKS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp

convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
is_clean_ecg = False
data_type = dr.DataTypes.ECG
measure = "rmssd"
# measure = "sdnn"
fs = 250.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK]
    for phase in phases:
        ha_ecg, la_ecg = preprocessing.load_data(task, data_type, phase, convert_sr, is_clean_ecg)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_ecg)):
            try:
                subject = la_ecg[i].iloc[0, 0]
                data = la_ecg[i].iloc[:, -1].to_numpy().flatten()
                if data.size == 0:
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
                else:
                    data = data.flatten()
                    ecg_features = preprocessing.get_ecg_metrics_pyhrv(data, fs)
            except Exception as e:
                print(f"LA index {i} has value None")
                value = np.asarray([0.0])

            value = ecg_features[measure]
            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_ecg)):
            try:
                subject = ha_ecg[i].iloc[0, 0]
                data = ha_ecg[i].iloc[:, -1].to_numpy().flatten()
                if data.size == 0:
                    print(f"HA index {i} has value None")
                    value = np.asarray([0.0])
                else:
                    data = data.flatten()
                    ecg_features = preprocessing.get_ecg_metrics_pyhrv(data, fs)
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = ecg_features[measure]
            value = np.insert(value, 0, subject)
            ha.append(value)

        # bpm_array = np.array([la, ha]).transpose()
        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


LA Baseline_Rest
HA Baseline_Rest
LA BugBox_Relax
HA BugBox_Relax
LA BugBox_Anticipate
HA BugBox_Anticipate
LA BugBox_Exposure
LA index 2 has value None
LA index 3 has value None
LA index 13 has value None
LA index 25 has value None
HA BugBox_Exposure
HA index 1 has value None
HA index 2 has value None
HA index 13 has value None
HA index 18 has value None
HA index 20 has value None
HA index 21 has value None
LA BugBox_Break
HA BugBox_Break
LA Speech_Relax
HA Speech_Relax
LA Speech_Anticipate
HA Speech_Anticipate
LA Speech_Exposure
HA Speech_Exposure
HA index 9 has value None
HA index 25 has value None
LA Speech_Break
HA Speech_Break


In [19]:
# FEATURE EXTRACTION - HF RR, LF RR
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
is_clean_ecg = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.ECG
measure = "hf_rr"
# measure = "lf_rr"
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK]
    for phase in phases:
        ha_ecg, la_ecg = preprocessing.load_data(task, data_type, phase, convert_sr, is_clean_ecg)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_ecg)):
            subject = la_ecg[i].iloc[0, 0]
            # try:
            data = la_ecg[i].iloc[:, -1]
            if data.size == 0:
                value = np.asarray([0.0])
            else:
                if measure == "hf_rr":
                    value = preprocessing.get_hf_rr(data)
                else:
                    value = preprocessing.get_lf_rr(data)
                    # print(value)
                if value is None: 
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
            # except Exception:
            #     print(f"LA index {i} has value None")
            #     value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_ecg)):
            subject = ha_ecg[i].iloc[0, 0]
            try:
                data = ha_ecg[i].iloc[:, -1]
                if data.size == 0:
                    value = np.asarray([0.0])
                else:
                    if measure == "hf_rr":
                        value = preprocessing.get_hf_rr(data)
                    else:
                        value = preprocessing.get_lf_rr(data)
                    if value is None: 
                        print(f"HA index {i} has value None")
                        value = np.asarray([0.0])
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            ha.append(value)

        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


LA Baseline_Rest
HA Baseline_Rest
LA BugBox_Relax
HA BugBox_Relax
LA BugBox_Anticipate
HA BugBox_Anticipate
LA BugBox_Exposure
HA BugBox_Exposure
LA BugBox_Break
HA BugBox_Break
LA Speech_Relax
HA Speech_Relax
LA Speech_Anticipate
HA Speech_Anticipate
LA Speech_Exposure
HA Speech_Exposure
LA Speech_Break
HA Speech_Break


In [7]:
# FEATURE EXTRACTION - EDA
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

from biosppy.signals import eda


convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.EDA
measure = "mean_SCL"
# measure = "SCR_rate"
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK]
    for phase in phases:
        ha_eda, la_eda = preprocessing.load_data(task, data_type, phase, convert_sr)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_eda)):
            try:
                subject = la_eda[i].iloc[0, 0]
                data = la_eda[i].iloc[:, -1]
                out = eda.eda(signal=data, sampling_rate=fs, show=False)
                data = out["filtered"]
                if measure == "mean_SCL":
                    value = preprocessing.get_mean_SCL(data)
                else:
                    value = preprocessing.get_SCR_rate(data)
                if value is None: 
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception:
                print(f"LA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_eda)):
            try:
                subject = ha_eda[i].iloc[0, 0]
                data = ha_eda[i].iloc[:, -1]
                out = eda.eda(signal=data, sampling_rate=fs, show=False)
                data = out["filtered"]
                if measure == "mean_SCL":
                    value = preprocessing.get_mean_SCL(data)
                else:
                    value = preprocessing.get_SCR_rate(data)
                if value is None: 
                    print(f"HA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            ha.append(value)

        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


LA Baseline_Rest
HA Baseline_Rest
LA BugBox_Relax
HA BugBox_Relax
LA BugBox_Anticipate
HA BugBox_Anticipate
LA BugBox_Exposure
LA index 2 has value None
LA index 3 has value None
LA index 13 has value None
LA index 25 has value None
HA BugBox_Exposure
HA index 1 has value None
HA index 2 has value None
HA index 13 has value None
HA index 18 has value None
HA index 20 has value None
HA index 21 has value None
LA BugBox_Break
HA BugBox_Break
LA Speech_Relax
HA Speech_Relax
LA Speech_Anticipate
HA Speech_Anticipate
LA Speech_Exposure
HA Speech_Exposure
HA index 9 has value None
HA index 25 has value None
LA Speech_Break
HA Speech_Break


In [82]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
task = dr.Tasks.BASELINE
phase = dr.Phases.BASE_REST
data_type = dr.DataTypes.ANKLE_L
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]

ha_acc, la_acc = preprocessing.load_data(task, data_type, phase, convert_sr)
print(ha_acc[0].shape)
print(ha_acc[0].head())

(7826, 9)
   subject         1         2         3         4         5         6  \
1     14.0  8.641993  1.985840 -4.641602  0.000000  0.732422  0.000000   
2     14.0  8.632421  1.966699 -4.670312  0.122070  0.610352 -0.061035   
3     14.0  8.622851  1.961914 -4.660742  0.244141  0.610352  0.000000   
4     14.0  8.608496  1.981055 -4.651172  0.305176  0.427246 -0.122070   
5     14.0  8.575000  1.981055 -4.651172  0.183105  0.366211 -0.061035   

            7          8  
1  157.153931 -59.403076  
2  157.142944 -59.419556  
3  157.137451 -59.430542  
4  157.137451 -59.441528  
5  157.131958 -59.452515  


In [ ]:
# FEATURE EXTRACTION - ANKLE ACC
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.ANKLE_L
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]
measure = "peak_ankle_acc_l"
# measure = "mean_ankle_activity_l"


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT]
    for phase in phases:
        ha_acc, la_acc = preprocessing.load_data(task, data_type, phase, convert_sr)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_acc)):
            try:
                subject = la_acc[i].iloc[0, 0]
                data = la_acc[i].iloc[:, 1:]
                # if measure == "peak_ankle_acc_r":
                if measure == "peak_ankle_acc_l":
                    value = preprocessing.get_peak_acc_value(data, acc_type="ankle")
                else:
                    value = preprocessing.get_mean_ankle_activity(data)
                if value is None: 
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception as e:
                print(f"LA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_acc)):
            try:
                subject = ha_acc[i].iloc[0, 0]
                data = ha_acc[i].iloc[:, 1:]
                # if measure == "peak_ankle_acc_r":
                if measure == "peak_ankle_acc_l":
                    value = preprocessing.get_peak_acc_value(data, acc_type="ankle")
                else:
                    value = preprocessing.get_mean_ankle_activity(data)
                if value is None: 
                    print(f"HA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            ha.append(value)

        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


In [ ]:
# FEATURE EXTRACTION - WRIST ACC
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.WRIST_L
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]
# measure = "peak_wrist_acc_l"
measure = "mean_wrist_activity_l"


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT]
    for phase in phases:
        ha_acc, la_acc = preprocessing.load_data(task, data_type, phase, convert_sr)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_acc)):
            try:
                subject = la_acc[i].iloc[0, 0]
                data = la_acc[i].iloc[:, 1:]
                # if measure == "peak_wrist_acc_r":
                if measure == "peak_wrist_acc_l":
                    value = preprocessing.get_peak_acc_value(data, acc_type="wrist")
                else:
                    value = preprocessing.get_mean_ankle_activity(data)
                if value is None: 
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception as e:
                print(f"LA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_acc)):
            try:
                subject = ha_acc[i].iloc[0, 0]
                data = ha_acc[i].iloc[:, 1:]
                # if measure == "peak_wrist_acc_r":
                if measure == "peak_wrist_acc_l":
                    value = preprocessing.get_peak_acc_value(data, acc_type="wrist")
                else:
                    value = preprocessing.get_mean_ankle_activity(data)
                if value is None: 
                    print(f"HA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            ha.append(value)

        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)


In [ ]:
# FEATURE EXTRACTION - TORSO
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.POSTURE
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]
# measure = "mean_posture"
measure = "mean_activity_level_torso"


for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT]
    for phase in phases:
        ha_posture, la_posture = preprocessing.load_data(task, data_type, phase, convert_sr)
        la = []
        ha = []
        print(f"LA {phase}")
        for i in range(len(la_posture)):
            try:
                subject = la_posture[i].iloc[0, 0]
                data = la_posture[i].iloc[:, 1:]
                if measure == "mean_posture":
                    value = preprocessing.get_mean_posture(data)
                else:
                    value = preprocessing.get_mean_activity_torso(data)
                if value is None: 
                    print(f"LA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception as e:
                print(f"LA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            la.append(value)

        print(f"HA {phase}")
        for i in range(len(ha_posture)):
            try:
                subject = ha_posture[i].iloc[0, 0]
                data = ha_posture[i].iloc[:, 1:]
                if measure == "mean_posture":
                    value = preprocessing.get_mean_posture(data)
                else:
                    value = preprocessing.get_mean_activity_torso(data)
                if value is None: 
                    print(f"HA index {i} has value None")
                    value = np.asarray([0.0])
            except Exception:
                print(f"HA index {i} has value None")
                value = np.asarray([0.0])

            value = np.insert(value, 0, subject)
            ha.append(value)

        ha_df = pd.DataFrame(ha)
        la_df = pd.DataFrame(la)

        ha_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_ha.csv")
        la_file = os.path.join(dr.Paths.METRICS, f"{measure}_{phase}_la.csv")
        with open(ha_file, "w+") as f:
            ha_df.to_csv(f)
        with open(la_file, "w+") as f:
            la_df.to_csv(f)
